# G only ACGT Probabilities

> 

In [ ]:
import os, json, re

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import hilbertcurve
from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna  import *
from EnvDL.dlfn import * # includes LSUV_

from tqdm import tqdm

In [ ]:
use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F # F.mse_loss

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger


device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.21_g2fc_G_ACGT_FCN_Res/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
ACGT = np.load(load_from+'ACGT.npy')

In [ ]:
# reduce Size of ACGT ----
# Since these are probabilities we can immediately save 1/4 of the space needed by using ACG instead of ACGT
ACGT = ACGT[:, 0:-1, :]

### Create train/test validate indicies from json

In [ ]:
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)

temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)

In [ ]:
train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

## One Hot Encoded

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
# Debugging Data -----------------------------------------------------------------------------------

train_idx = val_dict['train_idx']
test_idx  = val_dict['test_idx']

training_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(train_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

validation_dataloader = DataLoader(
    ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
                G = torch.from_numpy(ACGT).to(torch.float), 
                idx_original = torch.from_numpy(np.array(test_idx)),
                idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
                use_gpu_num = 0,
                device = 'cuda'
               ),
    batch_size = 50,
    shuffle = True
)

In [ ]:
# TODO, move this to a module
class Linear_res_block(nn.Module):
    def __init__(self, in_size, out_size, drop_pr):
        super(Linear_res_block, self).__init__()
        self.squish = nn.Linear(in_size, out_size)
        self.block1 = nn.Sequential(
            nn.Linear(out_size, out_size),
            nn.ReLU(),
            nn.Dropout(drop_pr)
        )
        self.block2 = nn.Sequential(
            nn.Linear(out_size, out_size),
            nn.ReLU(),
            nn.Dropout(drop_pr)
        )
    def forward(self, x):
        squish_residual = self.squish(x)
        out = self.block1(squish_residual)
        out = self.block2(out)
        out += squish_residual
        return out  

In [ ]:
# A quirk of this is that to get only a single layer the length of the input tensor must be passed in. for 2+ I'll figure it out.
class NeuralNetwork(nn.Module):
    def __init__(self, parameterization):
        super(NeuralNetwork, self).__init__()            
        module_list = []

        max_layer = parameterization['num_layers']
        for i in range(max_layer):
            if i  == 0:
                name_in = f"in_{i+1}_of_{max_layer}"
            else:
                name_in = f"out_{i}_of_{max_layer}"
            name_out = f"out_{i+1}_of_{max_layer}"
            name_drop= f"drop_{i+1}_of_{max_layer}"

            if i == 0:
                module_list += [nn.Flatten()]
            

            module_list += [
                Linear_res_block(
                    in_size  = parameterization[name_in], 
                    out_size = parameterization[name_out], 
                    drop_pr  = parameterization[name_drop])]
            
            if (i+1) == max_layer:
                module_list += [nn.Linear(parameterization[name_out], 1)]
                
        self.x_network = nn.ModuleList(module_list)
        
    def forward(self, x):
        
        for mod in self.x_network:
            if mod == self.x_network[-1]:
                out = x # get the penultimate layer's outputs for later
            x = mod(x)
        
        pred = x
        return pred, out

# model = NeuralNetwork(
#     parameterization = parameterization).to(device)

# model(next(iter(training_dataloader))[0][0:5])

In [ ]:
# convenince wrapper to fill in for R's seq or x:y notation
def linrange(start, stop):
    import numpy as np
    diff = start - stop
    res = np.linspace(start, stop, abs(diff)+1).astype(int)
    return res 

In [ ]:
ACGT.shape

(4926, 3, 125891)

In [ ]:
[2**i for i in linrange(11, 4)]

[2048, 1024, 512, 256, 128, 64, 32, 16]

In [ ]:
# This one is designed to go from (3x125891) -> ~1258.91  -> ~125.891 -> ~12.5891 -> 1
layer_sizes = [1024, 128, 12]
layer_drops = [0.1 for e in layer_sizes]

num_layers = len(layer_sizes)

params = {
    'num_layers':num_layers,
    f"in_1_of_{num_layers}": (ACGT.shape[1] * ACGT.shape[2])
}

for i in range(num_layers):
    params[f"out_{ i + 1}_of_{num_layers}"] = layer_sizes[i]
    params[f"drop_{ i + 1}_of_{num_layers}"] = layer_drops[i]
        

In [ ]:
model = NeuralNetwork(parameterization = params).to(device)
# model

In [ ]:
# LSUV_(model, data = next(iter(training_dataloader))[0] )

In [ ]:
# model(next(iter(training_dataloader))[0])

In [ ]:
# Module for training subnetworks.
class plDNN_ACGT(pl.LightningModule):
    def __init__(self, mod):
        super().__init__()
        self.mod = mod
        
    def training_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        with torch.no_grad():
            weight_list=[(name, param) for name, param in model.named_parameters() if name.split('.')[-1] == 'weight']
            for l in weight_list:
                self.log(("train_mean"+l[0]), l[1].mean())
                self.log(("train_std"+l[0]), l[1].std())        
        return(loss)
        
    def validation_step(self, batch, batch_idx):
        g_i, y_i = batch
        pred, out = self.mod(g_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
max_epoch = 100
DNNG = plDNN_ACGT(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-fcn-res")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | NeuralNetwork | 389 M 
---------------------------------------
389 M     Trainable params
0         Non-trainable params
389 M     Total params
1,556.014 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
torch.save(DNNG.mod, cache_path+'g-acgt-fcn-res'+'.pt')